In [1]:
import spotipy
import spotipy.util as util
import pandas as pd

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = 'ENTER KEY'
secret = 'ENTER KEY'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
playlist_creator = "kdmorris23"
playlist_id = "2cnBaMdriWPTEmX4octhJq" # Kanye Playlist

In [4]:
def get_playlist_tracks_more_than_100_songs(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
               'title':[playlist_tracks_titles],
               'all_artists':[playlist_tracks_artists],
               'popularity':[playlist_tracks_popularity],
               'release_date':[playlist_tracks_first_release_date],
               'danceability':[features[0]['danceability']],
               'energy':[features[0]['energy']],
               'key':[features[0]['key']],
               'loudness':[features[0]['loudness']],
               'mode':[features[0]['mode']],
               'acousticness':[features[0]['acousticness']],
               'instrumentalness':[features[0]['instrumentalness']],
               'liveness':[features[0]['liveness']],
               'valence':[features[0]['valence']],
               'tempo':[features[0]['tempo']],
               'duration_ms':[features[0]['duration_ms']],
               'time_signature':[features[0]['time_signature']]
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue
                
    return features_df

In [5]:
playlist_df = get_playlist_tracks_more_than_100_songs(playlist_creator, playlist_id)

In [11]:
playlist_df

,title,popularity,danceability,energy,loudness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,We Don't Care,66,0.595,0.754,-5.827,0.0142,0.000000,0.2470,0.647,83.155,239880,4
1,All Falls Down,80,0.657,0.734,-4.832,0.1490,0.000000,0.1390,0.434,91.030,223507,4
2,I'll Fly Away,55,0.239,0.250,-11.414,0.7600,0.000000,0.1240,0.564,172.338,69707,4
3,Spaceship,66,0.507,0.753,-5.310,0.5030,0.000000,0.5730,0.657,178.068,324227,3
4,Jesus Walks,72,0.637,0.834,-4.686,0.6140,0.000000,0.3170,0.715,87.312,193733,4
...,...,...,...,...,...,...,...,...,...,...,...,...
172,Jail pt 2,1,0.700,0.572,-6.459,0.4530,0.000000,0.0871,0.686,109.985,297000,4
173,Ok Ok pt 2,1,0.641,0.473,-9.068,0.4060,0.000002,0.1300,0.412,82.295,204630,4
174,Junya pt 2,1,0.753,0.767,-3.773,0.3090,0.000000,0.2790,0.258,148.130,182635,4
175,Jesus Lord pt 2,1,0.524,0.726,-6.628,0.0705,0.019200,0.0877,0.146,107.400,690732,5


In [10]:
playlist_df = playlist_df[['title', 'popularity','danceability', 'energy', 'loudness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature']]

In [12]:
playlist_df.to_csv('kanye_catalog_track_attributes.csv', index = False)